In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/1.item_number'):
    shutil.rmtree('./file/1.item_number')

os.makedirs('./file/1.item_number')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = input_.loc[a, 'Url']
            
            dict_param = {}
            list_param = url.split('?')[1].split('&') if '?' in url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]
            
            dict_param.pop('_pgn', '')
            dict_param['orig_cvip'] = 'true'
            dict_param['_ipg'] = '240'
            
            if url.startswith('https://www.ebay.com/'):
                dict_param['_ul'] = 'US'
                dict_param['_stpos'] = '91710'
            elif url.startswith('https://www.ebay.de/'):
                dict_param['_ul'] = 'DE'
                dict_param['_stpos'] = '10115'
            elif url.startswith('https://www.ebay.co.uk/'):
                dict_param['_ul'] = 'GB'
                dict_param['_stpos'] = 'BS81QU'
            elif url.startswith('https://www.ebay.com.au/'):
                dict_param['_ul'] = 'AU'
                dict_param['_stpos'] = '2019'
            elif url.startswith('https://www.ebay.ca/'):
                dict_param['_ul'] = 'CA'
                dict_param['_stpos'] = 'M5S2E8'
            
            url = url.split('?')[0] + '?'
            for key, value in dict_param.items():
                url += f'{key}={value}&'
        
            list_item_number, page = [], 1
            while True:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = f'''{url}_pgn={page}'''
                        
                        resp = requests.get(request_url,
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
    
                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))
            
                            dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')
            
                            list_item_number_temp = [w[2]['model']['action']['params']['itemId'].strip() if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params'] else '' for w in dict_['o']['w']]
                            if list_item_number_temp:
                                while '' in list_item_number_temp:
                                    list_item_number_temp.remove('')
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =
    
                list_item_number += list_item_number_temp
                list_item_number = sorted(list(set(list_item_number)), key=list_item_number.index)
                
                # = = = = = = = = = = = = = = =
    
                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
    
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{request_url} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
    
                # = = = = = = = = = = = = = = =
    
                page += 1
    
                # = = = = = = = = = = = = = = =
    
                if not html.xpath('//nav[@class="pagination"]/a[@type="next"]'):
                    break
                    
            # = = = = = = = = = = = = = = =
    
            if not list_item_number:
                raise
        
            # = = = = = = = = = = = = = = =

            output = pd.DataFrame({'No': [i+1 for i in range(len(list_item_number))],
                                   'Item Number': list_item_number})

            output = output.sort_values(by=['No'],
                                        ascending=[True],
                                        ignore_index=True)
            
            output.to_excel(f'''./file/1.item_number/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
            df_temp.loc[0, 'Request_Url'] = request_url
            
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/item_number_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.item_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/1.item_number/{file}',
              f'''./file/1.item_number/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[尝试次数：4] - 1.https://www.ebay.com/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=scitoo-autoparts&store_cat=0&store_name=cnbatteries&_oac=1&_nkw=ac%20compressor&orig_cvip=true&_ipg=240&_ul=US&_stpos=91710&_pgn=1 > Page 1
[剩余数量：0] - [当前时间：09:36:47]

[尝试次数：2] - 2.https://www.ebay.de/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=atp-autoteile&store_name=atpautoteile&_oac=1&_nkw=t%C3%BCrschloss&rt=nc&LH_BIN=1&orig_cvip=true&_ipg=240&_ul=DE&_stpos=10115&_pgn=1 > Page 1
[剩余数量：0] - [当前时间：09:36:48]

[尝试次数：3] - 2.https://www.ebay.de/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=atp-autoteile&store_name=atpautoteile&_oac=1&_nkw=t%C3%BCrschloss&rt=nc&LH_BIN=1&orig_cvip=true&_ipg=240&_ul=DE&_stpos=10115&_pgn=2 > Page 2
[剩余数量：0] - [当前时间：09:36:53]

[尝试次数：5] - 1.https://www.ebay.com/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=scitoo-autoparts&store_cat=0&store_name=cnbatteries&_oac=1&_nkw=ac%20compressor&orig_cvip=tru

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3823.43it/s]

Done ~
